# Preprocessing

Importing needed libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import joblib
from dask.distributed import LocalCluster, Client
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.base import TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, recall_score, precision_score, accuracy_score, f1_score

Initialize cluster

In [2]:
cluster = LocalCluster(n_workers = 4, threads_per_worker = 1, memory_limit = '6GB')
client = Client(cluster)

Reading csv file

In [3]:
df = pd.read_csv('datasets/Churn.csv', low_memory = False)
df.head()

,CustomerID,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,...,ReferralsMadeBySubscriber,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus
0,3000002,Yes,24.00,219.0,22.0,0.25,0.0,0.0,-157.0,-19.0,...,0,4,No,0,30,Yes,1-Highest,Suburban,Professional,No
1,3000010,Yes,16.99,10.0,17.0,0.00,0.0,0.0,-4.0,0.0,...,0,5,No,0,30,No,4-Medium,Suburban,Professional,Yes
2,3000014,No,38.00,8.0,38.0,0.00,0.0,0.0,-2.0,0.0,...,0,6,No,0,Unknown,No,3-Good,Town,Crafts,Yes
3,3000022,No,82.28,1312.0,75.0,1.24,0.0,0.0,157.0,8.1,...,0,6,No,0,10,No,4-Medium,Other,Other,No
4,3000026,Yes,17.14,0.0,17.0,0.00,0.0,0.0,0.0,-0.2,...,0,9,No,1,10,No,1-Highest,Other,Professional,Yes


Checking dtypes

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71047 entries, 0 to 71046
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CustomerID                 71047 non-null  int64  
 1   Churn                      51047 non-null  object 
 2   MonthlyRevenue             70831 non-null  float64
 3   MonthlyMinutes             70831 non-null  float64
 4   TotalRecurringCharge       70831 non-null  float64
 5   DirectorAssistedCalls      70831 non-null  float64
 6   OverageMinutes             70831 non-null  float64
 7   RoamingCalls               70831 non-null  float64
 8   PercChangeMinutes          70545 non-null  float64
 9   PercChangeRevenues         70545 non-null  float64
 10  DroppedCalls               71047 non-null  float64
 11  BlockedCalls               71047 non-null  float64
 12  UnansweredCalls            71047 non-null  float64
 13  CustomerCareCalls          71047 non-null  flo

Verifying null values

In [5]:
# grabbing all missing values
total_missing = df.isnull().sum().reset_index().rename(columns = {'index': 'feature', 0: 'missing_count'})

# selecting ones whic have missing_count greater than 0
actual_missing = total_missing[total_missing.missing_count > 0].copy().reset_index(drop = True)

# showing in terms of relative percentages 
actual_missing['missin_count_relavite'] = actual_missing['missing_count'] / len(df)

# adding its dtypes
dtype_dict = df.dtypes.to_dict()
actual_missing['dtype'] = actual_missing.feature.map(dtype_dict)

# missing dataframe
actual_missing

,feature,missing_count,missin_count_relavite,dtype
0,Churn,20000,0.281504,object
1,MonthlyRevenue,216,0.003040,float64
2,MonthlyMinutes,216,0.003040,float64
3,TotalRecurringCharge,216,0.003040,float64
4,DirectorAssistedCalls,216,0.003040,float64
5,OverageMinutes,216,0.003040,float64
6,RoamingCalls,216,0.003040,float64
7,PercChangeMinutes,502,0.007066,float64
8,PercChangeRevenues,502,0.007066,float64
9,ServiceArea,28,0.000394,object


As it can be seen, there are 20000 missing values on the target "Churn", this group is stored in another datadrame for future purposes, but as first step we clean the null values by dropping their rows cause they represent fewer than 2%

In [6]:
# drop null values
values_to_drop = actual_missing.feature.tolist()[1:]

df = df.dropna(subset = values_to_drop, axis = 0)

# getting those
df_no_target = df[df.Churn.isnull()].copy().reset_index(drop = True)

# dropping 20000 nan values in the original dataframe
df = df.dropna(subset = 'Churn')

# removing churn feature from actual_missing
actual_missing = actual_missing[actual_missing.feature != 'Churn']

# verifying new shape
df.shape

(49752, 58)

Verifying null values

In [7]:
df.isnull().sum().sum()

0

Labeling target

In [8]:
df['Churn'] = np.where(df['Churn'] == 'Yes', 1, 0)

Creating a class to clean outliers

In [9]:
class CleanOutliers(TransformerMixin):
    def __init__(self):
        '''
        Treat outliers
        '''
    def set_output(self, transform = 'pandas'):
        return self

    def _get_boundaries(self, X_feature):
        q1 = X_feature.quantile(0.25)
        q3 = X_feature.quantile(0.75)
        irs = q3 - q1
        lower_bound = q1 - 1.5 * irs
        upper_bound = q3 + 1.5 * irs
        return lower_bound, upper_bound

    def fit(self, X, y = None):
        self.boundary = {column: self._get_boundaries(X[column]) for column in X.columns}
        return self

    def transform(self, X, y = None):
        X_gei = X.copy()
        current_columns = X_gei.columns.tolist()
        for column in current_columns:
            lower_bound, upper_bound = self.boundary[column]
            X_gei[column] = X_gei[column].clip(lower = lower_bound,
                                        upper = upper_bound)
        return X_gei.copy()

Verfying categorical values before performing encoding

In [10]:
cat_features = df.describe(include = object).T
cat_features_to_regroup = cat_features[cat_features.unique > 4]
cat_features_to_regroup

,count,unique,top,freq
ServiceArea,49752,743,NYCBRO917,1646
HandsetPrice,49752,16,Unknown,28263
CreditRating,49752,7,2-High,18692
Occupation,49752,8,Other,36453


Selecting those columns to check on its values

In [11]:
df_cat_features_to_regroup = df[['ServiceArea', 'HandsetPrice', 'CreditRating', 'Occupation']]
df_cat_features_to_regroup

,ServiceArea,HandsetPrice,CreditRating,Occupation
0,SEAPOR503,30,1-Highest,Professional
1,PITHOM412,30,4-Medium,Professional
2,MILMIL414,Unknown,3-Good,Crafts
3,PITHOM412,10,4-Medium,Other
4,OKCTUL918,10,1-Highest,Professional
...,...,...,...,...
51035,NEVELC619,Unknown,2-High,Professional
51037,NEVENC760,Unknown,3-Good,Other
51040,LAXVNY818,Unknown,1-Highest,Professional
51041,LAXDOW562,30,5-Low,Professional


Value counts per cat feature

In [12]:
df.ServiceArea.value_counts()

ServiceArea
NYCBRO917    1646
DALDAL214    1479
HOUHOU281    1471
NYCMAN917    1156
DALFTW817     771
             ... 
SANROM956       1
INDCRA765       1
SEWSUN509       1
ATLDBL478       1
NCRDNN910       1
Name: count, Length: 743, dtype: int64

In [13]:
df.HandsetPrice.value_counts()

HandsetPrice
Unknown    28263
30          7147
150         4024
130         2043
80          1901
10          1869
60          1730
200         1227
100         1209
40           240
400           45
250           20
300           12
180            9
500            7
240            6
Name: count, dtype: int64

Creating a function to regroup specific features

In [14]:
df.HandsetPrice.unique()

array(['30', 'Unknown', '10', '80', '150', '300', '40', '200', '100',
       '130', '60', '400', '240', '250', '180', '500'], dtype=object)

In [15]:
def regroup_cat_values(data):
    # ServiceArea
    top_4 = df.ServiceArea.value_counts().head(4).index.tolist()
    data['ServiceArea'] = np.where(data['ServiceArea'].isin(top_4), 
                                    data['ServiceArea'],
                                    'OTHER')
    # HandsetPrice
    data['HandsetPrice'] = np.where(data['HandsetPrice'] == 'Unknown', '30', data['HandsetPrice'])
    data['HandsetPrice'] = data['HandsetPrice'].astype('int16')

    return data

Apply the function to both dataframes

In [16]:
df = regroup_cat_values(df)
df_no_target = regroup_cat_values(df_no_target)

In [17]:
df.describe(include = object).T

,count,unique,top,freq
ServiceArea,49752,5,OTHER,44000
ChildrenInHH,49752,2,No,37483
HandsetRefurbished,49752,2,No,42852
HandsetWebCapable,49752,2,Yes,44905
TruckOwner,49752,2,No,40280
RVOwner,49752,2,No,45619
Homeownership,49752,2,Known,33725
BuysViaMailOrder,49752,2,No,31432
RespondsToMailOffers,49752,2,No,30667
OptOutMailings,49752,2,No,49006


It is observed that can be label encoded is CreditRating

In [18]:
df.CreditRating.unique()

array(['1-Highest', '4-Medium', '3-Good', '2-High', '5-Low', '6-VeryLow',
       '7-Lowest'], dtype=object)

Creating a local class for label encoding

Splitting dataframe

In [19]:
# Getting X and y matrices
X = df.drop(columns = ['CustomerID','Churn'])
y = df.Churn

# splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state = 24)

Performing the final treatment

In [20]:
# selecting dtypes
num_col = X_train.select_dtypes(include = np.number).columns.tolist()
cat_col_ohe = X_train.select_dtypes(include = ['object', 'category']).columns.tolist()
#cat_col_ohe.remove('CreditRating')
cat_col_lab = 'CreditRating'
# creating pipelines or transformers
num_prep = Pipeline([('outlier', CleanOutliers().set_output(transform ='pandas')), 
                    ('impute', SimpleImputer(strategy = 'mean').set_output(transform = 'pandas'))])

cat_prep_ohe = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore').set_output(transform ='pandas')
#cat_prep_lab = LabelEncoder().set_output(transform = 'pandas')

# creating transformer
column_transformer = ColumnTransformer([('num', num_prep, num_col),
                                        ('ohe', cat_prep_ohe, cat_col_ohe)],
                                        remainder = 'passthrough')\
                                    .set_output(transform = 'pandas')

# testing transformer
X_train_tr = column_transformer.fit_transform(X_train)

# Data Modelling

Creating a function to evaluate performance

In [21]:
def evaluation_metric(model_name, y_train_probs, y_test_probs, y_train_actual, y_train_pred, y_test_actual, y_test_pred):
    roc_auc_train = roc_auc_score(y_train_actual, y_train_probs)
    roc_auc_test = roc_auc_score(y_test_actual, y_test_probs)

    gini_train = 2 * roc_auc_train - 1
    gini_test = 2 * roc_auc_test - 1

    recall_train = recall_score(y_train_actual, y_train_pred)
    recall_test = recall_score(y_test_actual, y_test_pred)

    precision_train = precision_score(y_train_actual, y_train_pred)
    precision_test = precision_score(y_test_actual, y_test_pred)

    f1_train = f1_score(y_train_actual, y_train_pred)
    f1_test = f1_score(y_test_actual, y_test_pred)

    accuracy_train = accuracy_score(y_train_actual, y_train_pred)
    accuracy_test = accuracy_score(y_test_actual, y_test_pred)

    # Table creation
    results_dict = {f'{model_name} Train set': [roc_auc_train, gini_train, recall_train, precision_train, f1_train, accuracy_train],
                f'{model_name} Test set': [roc_auc_test, gini_test, recall_test, precision_test, f1_test, accuracy_test]}

    results = pd.DataFrame(results_dict, index = ['ROC AUC Score', 'Gini Score', 'Recall Score', 'Precision Score',
                                                'F1 Score', 'Accuracy Score'])
    return results

Creating base models

In [37]:
# classifiers and scaler
scaler = StandardScaler()
lr = LogisticRegression(class_weight = 'balanced', random_state = 24, n_jobs = -1)
#svc = SVC(probability = True, random_state = 24)
knn = KNeighborsClassifier(weights = 'distance', n_jobs = -1)
dtc = DecisionTreeClassifier(class_weight = 'balanced', random_state = 24)
rfc = RandomForestClassifier(class_weight = 'balanced', random_state = 24)
xgb = XGBClassifier(scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1]), random_state = 24, n_jobs = -1)

# declaring pipelines
lr_pipe = Pipeline([('prep', column_transformer), ('sc', scaler), ('clf', lr)])
#svc_pipe = Pipeline([('prep', column_transformer), ('sc', scaler), ('clf', svc)])
knn_pipe = Pipeline([('prep', column_transformer), ('sc', scaler), ('clf', knn)])
dtc_pipe = Pipeline([('prep', column_transformer), ('sc', scaler), ('clf', dtc)])
rfc_pipe = Pipeline([('prep', column_transformer), ('sc', scaler), ('clf', rfc)])
xgb_pipe = Pipeline([('prep', column_transformer), ('sc', scaler), ('clf', xgb)])

# fitting pipelines
with joblib.parallel_backend('dask', scatter = (X_train, y_train)):
    lr_pipe.fit(X_train, y_train)
    #svc_pipe.fit(X_train, y_train)
    knn_pipe.fit(X_train, y_train)
    dtc_pipe.fit(X_train, y_train)
    rfc_pipe.fit(X_train, y_train)
    xgb_pipe.fit(X_train, y_train)

Base metrics

In [38]:
total_metrics = pd.DataFrame()

pipes = [('LR', lr_pipe), ('KNN', knn_pipe), ('DTC', dtc_pipe), ('RFC', rfc_pipe), ('XGB', xgb_pipe)]

for clf_name, clf in pipes:
    y_train_pred_probs = clf.predict_proba(X_train)[:, 1]
    y_test_pred_probs = clf.predict_proba(X_test)[:, 1]

    y_train_pred_class = clf.predict(X_train)
    y_test_pred_class = clf.predict(X_test)

    result = evaluation_metric(clf_name, y_train_pred_probs, y_test_pred_probs,
                                y_train, y_train_pred_class, y_test, y_test_pred_class)
    
    total_metrics = pd.concat([total_metrics, result], axis = 1)

total_metrics

,LR Train set,LR Test set,KNN Train set,KNN Test set,DTC Train set,DTC Test set,RFC Train set,RFC Test set,XGB Train set,XGB Test set
ROC AUC Score,0.627440,0.620649,1.0,0.538532,1.0,0.543370,1.0,0.664735,0.926962,0.662638
Gini Score,0.254880,0.241299,1.0,0.077064,1.0,0.086740,1.0,0.329471,0.853925,0.325275
Recall Score,0.591966,0.587553,1.0,0.168776,1.0,0.346343,1.0,0.050281,0.867205,0.521449
Precision Score,0.364909,0.369200,1.0,0.328992,1.0,0.348057,1.0,0.588477,0.664181,0.417629
F1 Score,0.451499,0.453460,1.0,0.223100,1.0,0.347198,1.0,0.092647,0.752235,0.463800
Accuracy Score,0.588000,0.595217,1.0,0.664054,1.0,0.627776,1.0,0.718521,0.836361,0.655412
